# Boltzmann Machines

Two cases here: 
1. One is if a user will like a product or not (Yes/No)
2. The second is what the user would rate the product

These are the most common uses in practice.

Using the [MovieLens](https://grouplens.org/datasets/movielens/) dataset

## Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Data Preprocessing

In [2]:
movies = pd.read_csv('data/rbm/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1') # :: is the separator in the file

In [3]:
users = pd.read_csv('data/rbm/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1') # :: is the separator in the file

In [4]:
ratings = pd.read_csv('data/rbm/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

### Create the training and test sets

In [5]:
training_set = pd.read_csv('data/rbm/ml-100k/u1.base', delimiter = '\t')

In [6]:
training_set = np.array(training_set, dtype = 'int')

In [7]:
test_set = pd.read_csv('data/rbm/ml-100k/u1.test', delimiter = '\t')

In [8]:
test_set = np.array(test_set, dtype = 'int')

### Getting the number of users and movies

In [9]:
# We need to get the largest user id and movie id so it works for training and testing for any of the datasets
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0]))) 
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

### Convert the data into an array with users in rows and movies in columns

In [10]:
# We will have a list of lists that has every user and every movie so they will always be the same size
# Torch expects a list of lists

def convert(data):
    new_data = []
    for id_user in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_user] # Get all movies
        id_ratings = data[:,2][data[:,0] == id_user] # Get all ratings
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [11]:
# Example final state
# User 1 - [0,1,1,2]
# User 2 - [1,1,0,2]
# User 3 - [1,0,0,2]

training_set_converted = convert(training_set)
test_set_converted = convert(test_set)

### Converting the data into Torch tensors

In [12]:
training_set_torch = torch.FloatTensor(training_set_converted) # Expects the list of lists
test_set_torch = torch.FloatTensor(test_set_converted)

### Converting the ratings in to binary ratings 1 (Liked) or 0 (Not Liked)

In [13]:
training_set_torch[training_set_torch == 0]= -1 # Anything that doesn't have a rating was not reviewed (base) 
training_set_torch[training_set_torch == 1]= 0
training_set_torch[training_set_torch == 2]= 0
training_set_torch[training_set_torch >= 3]= 1

In [14]:
test_set_torch[test_set_torch == 0]= -1 # Anything that doesn't have a rating was not reviewed (base) 
test_set_torch[test_set_torch == 1]= 0
test_set_torch[test_set_torch == 2]= 0
test_set_torch[test_set_torch >= 3]= 1

## Create the Neural Network

In [15]:
class RBM():
    def __init__(self, nv, nh): # number of visible and number of hidden nodes
        self.W = torch.randn(nv, nh) # initialize the weights        
        self.a = torch.randn(1, nh)    # bias
        self.b = torch.randn(1, nv)    # batch
    def sample_hidden(self, x):
        wx = torch.mm(x, self.W) # makes product of 2 torch tensors
        activation = wx + self.a.expand_as(wx) # expand the bias (calculates the probability of the cell being acviated
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_visible(self, y): 
        wy = torch.mm(y, self.W.t())
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk): #v0 is initial vector, vk is visible nodes after k sampling, ph0 is hidden nodes initially, phk is hidden nodes after k sampling
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)       

In [16]:
nv = len(training_set_torch[0])
nh = 100 # Trying for 100 features, this can be experimented with
batch_size = 100

rbm = RBM(nv, nh)

## Training the RBM

In [17]:
nb_epoch = 100
#for epoch in range(1, nb_epoch + 1):
epoch = 0
history = []

# To find when the loss function plateaus
def is_plateauing(y_values, threshold=0.01, window=5, min_values=10): 
    if len(y_values) > min_values:
        diffs = np.diff(y_values)  # First derivative
        recent_diffs = diffs[-window:]  # Last 'window' slopes
        return np.all(np.abs(recent_diffs) < threshold)
    else:
        return False

while is_plateauing(history) is False: 
    epoch += 1
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set_torch[id_user:id_user+batch_size]
        v0 = training_set_torch[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_hidden(v0)
        for k in range(10):
            _, hk = rbm.sample_hidden(vk)
            _, vk = rbm.sample_visible(hk)
            vk[v0 < 0] = v0[v0 < 0] # make sure empty ratings remain -1
        phk,_ = rbm.sample_hidden(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.

    history.append(train_loss/s)
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))
    
    
        

epoch: 1 loss: tensor(0.3548)
epoch: 2 loss: tensor(0.2455)
epoch: 3 loss: tensor(0.2511)
epoch: 4 loss: tensor(0.2507)
epoch: 5 loss: tensor(0.2472)
epoch: 6 loss: tensor(0.2539)
epoch: 7 loss: tensor(0.2442)
epoch: 8 loss: tensor(0.2495)
epoch: 9 loss: tensor(0.2487)
epoch: 10 loss: tensor(0.2484)
epoch: 11 loss: tensor(0.2478)


## Testing the RBM

In [18]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set_torch[id_user:id_user+1] # 1 since doing one user at a time... remains training to activate the neurons for the test set
    vt = test_set_torch[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_hidden(v)
        _, v = rbm.sample_visible(h)            
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.
print('accuracy: ' + str((1-test_loss/s)*100) + "%")

    

accuracy: tensor(73.8039)%
